<a href="https://colab.research.google.com/github/asdfasdasd314/satire-news-detector/blob/main/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikeras
!pip install scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 55.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
hdbscan 0.8.41 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy # Potentially leave this out
import tensorflow as tf
# import sklearn
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from scikeras.wrappers import KerasClassifier
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
df = pd.read_csv("/content/drive/MyDrive/maker-portfolio-data/onion-data/OnionOrNot.csv")

# Drop 6000 random rows to check if there is a problem with unbalanced data
# class_0 = df[df['label'] == 0]

# to_drop = class_0.sample(n=6000, random_state=42)

# df_reduced = df.drop(to_drop.index)
# df = df_reduced
# -------------------------------------------------------------------------

df.head()
df.columns
df['label'].value_counts()

,count
label,
0,15000
1,9000


In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
def get_nlps(data: pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: nlp(x))

def count_pronouns(data: pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: len([ent.label_ for ent in x.ents if ent.label_ in ["PERSON", "ORG", "GPE"]]))

def count_people(data: pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: len([ent.text for ent in x.ents if ent.label_ == "PERSON"]))

def count_orgs(data: pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: len([ent.text for ent in x.ents if ent.label_ == "ORG"]))

def count_gpes(data: pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: len([ent.text for ent in x.ents if ent.label_ == "GPE"]))

def count_verbs(data: pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: len([token for token in x if token.pos_ == "VERB"]))

def count_auxiliaries(data: pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: len([token for token in x if token.pos_ == "AUX"]))

def count_adjectives(data: pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: len([token for token in x if token.pos_ == "ADJ"]))

def count_characters(data: pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: len(x))

def count_tokens(data: pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: len([ent.text for ent in x.ents]))

def count_exclamation(data:pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: len([token for token in x if token.pos_ == "!"]))

def count_quotes(data: pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: len([token for token in x if token.pos_ == "'"]))

def count_question_marks(data: pd.Series):
  copy = data.copy()
  return copy.apply(lambda x: len([token for token in x if token.pos == "?"]))

def vectorizer_preprocessor(text):
    text = text.lower()
    text = regex.sub('', text)  # remove unwanted punctuation
    return text

punct_to_remove = string.punctuation.replace('"', '').replace(',', '')
regex = re.compile(f"[{re.escape(punct_to_remove)}]")

# Token pattern: allow words with apostrophes (like don't), quotes, and commas
# This pattern:
# - Matches words with optional internal apostrophes, commas, and double quotes
# - E.g., it keeps: don't, "hello", new,york
token_pattern = r'\b[a-zA-Z0-9"\',]{2,}\b'

In [32]:
X = df['text']
y = df['label']

X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
vectorizer = CountVectorizer(stop_words="english", preprocessor=vectorizer_preprocessor, token_pattern=token_pattern)
vectorizer.fit_transform(X_train_text)

X_train_nlps, X_test_nlps = get_nlps(X_train_text), get_nlps(X_test_text)

In [34]:
X_test_text[X_test_text.isna()]

,text


In [20]:
X_train_pronouns, X_test_pronouns = count_pronouns(X_train_nlps), count_pronouns(X_test_nlps)
X_train_people, X_test_people = count_people(X_train_nlps), count_people(X_test_nlps)
X_train_orgs, X_test_orgs = count_orgs(X_train_nlps), count_orgs(X_test_nlps)
X_train_gpes, X_test_gpes = count_gpes(X_train_nlps), count_gpes(X_test_nlps)
X_train_verbs, X_test_verbs = count_verbs(X_train_nlps), count_verbs(X_test_nlps)
X_train_auxiliaries, X_test_auxiliaries = count_auxiliaries(X_train_nlps), count_auxiliaries(X_test_nlps)
X_train_adjectives, X_test_adjectives = count_adjectives(X_train_nlps), count_adjectives(X_test_nlps)
X_train_characters, X_test_characters = count_characters(X_train_text), count_characters(X_test_text)
X_train_tokens, X_test_tokens = count_tokens(X_train_nlps), count_tokens(X_test_nlps)
X_train_exclamation, X_test_exclamation = count_exclamation(X_train_nlps), count_exclamation(X_train_nlps)
X_train_quotes, X_test_quotes = count_quotes(X_train_nlps), count_quotes(X_train_nlps)
X_train_question_marks, X_test_question_marks = count_question_marks(X_train_nlps), count_question_marks(X_train_nlps)

In [24]:
train_df = pd.DataFrame({
    'text': X_train_text,
    'num_pronouns': X_train_pronouns,
    'num_characters': X_train_characters,
    'num_tokens': X_train_tokens,
    'num_verbs': X_train_verbs,
    'num_auxiliaries': X_train_auxiliaries,
    'num_adjectives': X_train_adjectives,
    'num_people': X_train_people,
    'num_orgs': X_train_orgs,
    'num_gpes': X_train_gpes,
    'num_exclamation': X_train_exclamation,
    'num_quotes': X_train_quotes,
    'num_question_marks': X_train_question_marks,
    'label': y_train
})

test_df = pd.DataFrame({
    'text': X_test_text,
    'num_pronouns': X_test_pronouns,
    'num_characters': X_test_characters,
    'num_tokens': X_test_tokens,
    'num_verbs': X_test_verbs,
    'num_auxiliaries': X_test_auxiliaries,
    'num_adjectives': X_test_adjectives,
    'num_people': X_test_people,
    'num_orgs': X_test_orgs,
    'num_gpes': X_test_gpes,
    'num_exclamation': X_test_exclamation,
    'num_quotes': X_test_quotes,
    'num_question_marks': X_test_question_marks,
    'label': y_test
})

In [25]:
num_features = len(vectorizer.vocabulary_) + train_df.shape[1] - 2

def make_model():
    model = keras.Sequential([
        layers.Input(shape=(num_features,)),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
    return model

# ColumnTransformer to handle different types of features
preprocessor = ColumnTransformer(
    [
      ('tfidf', vectorizer, 'text'),
      ('scaler', StandardScaler(), ['num_pronouns', 'num_characters', 'num_tokens', 'num_verbs', 'num_auxiliaries', 'num_adjectives', 'num_people', 'num_orgs', 'num_gpes', 'num_exclamation', 'num_quotes', 'num_question_marks']),  # scale numeric features
    ],
  )

# Combine with estimator
pipeline = Pipeline([
    ('features', preprocessor),
    ('classifier', KerasClassifier(model=make_model, epochs=2, batch_size=32))
])

In [26]:
pipeline.fit(train_df.drop(columns=['label']), train_df['label'])

Epoch 1/2
600/600 ━━━━━━━━━━━━━━━━━━━━ 36s 57ms/step - accuracy: 0.7808 - loss: 0.4961
Epoch 2/2
600/600 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.9312 - loss: 0.1953


Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('tfidf',
                                                  CountVectorizer(preprocessor=<function vectorizer_preprocessor at 0x7975ce9aa0c0>,
                                                                  stop_words='english',
                                                                  token_pattern='\\b[a-zA-Z0-9"\\\',]{2,}\\b'),
                                                  'text'),
                                                 ('scaler', StandardScaler(),
                                                  ['num_pronouns',
                                                   'num_characters',
                                                   'num_tokens', 'num_verbs',
                                                   'num_auxiliaries',
                                                   'num_adjectives',
                                                   'num_people', 'num_orgs',
                                                   'num_gpes',
                                                   'num_exclamation',
                                                   'num_quotes',
                                                   'num_question_marks'])])),
                ('classifier',
                 KerasClassifier(batch_size=32, epochs=2, model=<function make_model at 0x7975cc81c040>))])

In [27]:
y_pred = pipeline.predict(test_df.drop(columns=['label']))

print(accuracy_score(test_df['label'], y_pred))
print(precision_score(test_df['label'], y_pred))
print(recall_score(test_df['label'], y_pred))
print(confusion_matrix(test_df['label'], y_pred))

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
# right = df[pipeline.predict(vectorizer.transform(df['text'])) == y]
# wrong = df[pipeline.predict(vectorizer.transform(df['text'])) != y]

true = df[df['label'] == 0]
false = df[df['label'] == 1]

# right_test = right[right['text'].isin(X_test)]
# wrong_test = wrong[wrong['text'].isin(X_test)]

# Write to CSV to investigate what is going wrong (perhaps there's a pattern?)
# right.to_csv("/content/drive/MyDrive/onion/right.csv")
# wrong.to_csv("/content/drive/MyDrive/onion/wrong.csv")

# right_test.to_csv("/content/drive/MyDrive/onion/right_test.csv")
# wrong_test.to_csv("/content/drive/MyDrive/onion/wrong_test.csv")

NameError: name 'df' is not defined

In [ ]:
true_nlps = get_nlps(true['text'])
false_nlps = get_nlps(false['text'])

In [ ]:
true_entities = count_gpes(true_nlps)
false_entities = count_gpes(false_nlps)

total_true = 0
total_false = 0
for i in true_entities:
  total_true += i
for i in false_entities:
  total_false += i

print(total_true / 15000)
print(total_false / 9000)
print(true_entities)
print(false_entities)

Statistics:

True articles contain 8114 proper nouns and false articles contain 6104 proper nouns

True articles contain on average 0.29 mentions of GPEs and false contain 0.047 mentions (the differences in the other two types of entities are basically negligible)

True articles contain on average 1.906 verbs and fake articles contain 1.744 verbs

True articles contain on average 0.4084 auxiliary verbs (e.g. is, was, am) and false articles contain 0.5467 on average

True articles contain on average 0.7764 adjectives and false articles contain on average 0.522 adjectives

True articles are on average 71.62 characters long and false articles are on average 83.26 characters long

True articles are on average 13.35 tokens long and false articles are 15.34 tokens long

True articles are on average 11.69 words long and false articles are on average 13.703 words long (using `.split(' ')`)



These results show that there is fundamentally something in that data that allows the model to distinguish between fake news and real news reliably, although whether or not we will be able to make it to 100% is the tricky part

**Potential Hypothesis**: The vectorizer we are currently using picks up on rarely used words, and Onion articles often use crazy words, for example: "*Environmental Win: Scientists Just Cut Down The World’s Oldest Tree After It Went Over 5,000 Years Without Giving Us A Single Goddam Apple*"

So roughly 80% of the time looking for crazy words or strings of crazy words will work, whereas this (the model got it wrong): "*ICE Sends Agents Home With Sacks Of Flour To Practice What It Like Detaining Real Baby*" contains no crazy words and is only crazy given the relation of "*ICE agents*" to "*detaining real baby*" which maybe the model has a harder time picking up on? So potentially needing more data or something like an LLM that more easily picks up on the meanings of words in relation to each other

In [ ]:
# To test this hypothesis, look for any trends in the vectorized data
right_vecs = vectorizer.transform(right['text'])
wrong_vecs = vectorizer.transform(wrong['text'])
true_vecs = vectorizer.transform(true['text'])
false_vecs = vectorizer.transform(false['text'])

right_pred_word_occurrences = pd.DataFrame(right_vecs.toarray())
wrong_pred_word_occurrences = pd.DataFrame(wrong_vecs.toarray())
true_pred_word_occurrences = pd.DataFrame(true_vecs.toarray())
false_pred_word_occurrences = pd.DataFrame(false_vecs.toarray())

right_vec_sums = np.sum(right_pred_word_occurrences, axis=0)
wrong_vec_sums = np.sum(wrong_pred_word_occurrences, axis=0)
true_vec_sums = np.sum(true_pred_word_occurrences, axis=0)
false_vec_sums = np.sum(false_pred_word_occurrences, axis=0)

lower_bound = 1
upper_bound = 20

most_frequent_rights = right_vec_sums.sort_values(ascending=False)[lower_bound:upper_bound,]
most_frequent_wrongs = wrong_vec_sums.sort_values(ascending=False)[lower_bound:upper_bound,]
most_frequent_trues = true_vec_sums.sort_values(ascending=False)[lower_bound:upper_bound,]
most_frequent_falses = false_vec_sums.sort_values(ascending=False)[lower_bound:upper_bound,]

# vocabulary_ maps: word → index
vocab = vectorizer.vocabulary_

# Reverse it: index → word
idx_to_word = {index: word for word, index in vocab.items()}

right_labels = pd.Series([idx_to_word[i] for i in most_frequent_rights.keys()])
wrong_labels = pd.Series([idx_to_word[i] for i in most_frequent_wrongs.keys()])
true_labels = pd.Series([idx_to_word[i] for i in most_frequent_trues.keys()])
false_labels = pd.Series([idx_to_word[i] for i in most_frequent_falses.keys()])

In [ ]:
# Create 1 row, 2 columns of subplots
fig, axes = plt.subplots(2, 2, figsize=(10, 5))

# Subplot 1
axes[0, 0].bar(right_labels, most_frequent_rights, color='skyblue')
axes[0, 0].set_title('Right Predictions')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_xticklabels(right_labels, rotation=50)

# Subplot 2
axes[0, 1].bar(wrong_labels, most_frequent_wrongs, color='lightgreen')
axes[0, 1].set_title('Wrong Predictions')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_xticklabels(wrong_labels, rotation=50)

# Subplot 3
axes[1, 0].bar(true_labels, most_frequent_trues, color='yellow')
axes[1, 0].set_title('True Headlines')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_xticklabels(true_labels, rotation=50)

# Subplot 4
axes[1, 1].bar(false_labels, most_frequent_falses, color='lightpink')
axes[1, 1].set_title('False Headlines')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_xticklabels(false_labels, rotation=50)

# Optional: Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# model.predict(vectorizer.transform(["Something Forbidden Stirs Deep Within Trump After He Sees Political Cartoon Depicting Him As Chicken"]))
model.predict(vectorizer.transform(["Bold Move: Hulu Has Announced That They’re Gonna Go Ahead And Reboot ‘Shrill’ While It’s Still On Since You Idiots Will Watch Anything"]))